In [1]:
import sys
sys.path.insert(0, 'c:\\users\\emrek\\source\\guidance\\prompt-plan\\promptlib\\')

In [2]:
from promptlib import PromptNode
from promptlib import set_model, append, gen, choose, begin, end
from promptlib.models import LLM, TransformersLLM
from promptlib.constraints import Constraint, DummyCharacterConstraint, OddEvenDigitConstraint, ContextFreeGrammarConstraint

C:\Users\emrek\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Demonstrate a dummy constraint

In [3]:
llm = TransformersLLM("gpt2")

pn = PromptNode().set_model(llm)              \
    .append("My computer says Hello")                        \
    .constrain(DummyCharacterConstraint(positiveConstraint=['o','r','l','d','w',' ', '!'])) \
    .gen(max_tokens=10)

text = pn.get_all_text()
print(text)

My computer says Hello! or doododododododod


## Constraints are scoped to the current begin/end block

In [4]:
llm = TransformersLLM("gpt2")

pn = PromptNode().set_model(llm)              \
    .append("Hello world!")                   \
    .begin()                                  \
    .append("I sing the ")                    \
    .constrain(DummyCharacterConstraint(positiveConstraint=['A', 'B', 'C', 'a', 'b', 'c', 's'])) \
    .gen(max_tokens=10)                       \
    .end()                                    \
    .append(" song.")                         \
    .gen(max_tokens=10)

text = pn.get_all_text()
print(text)

Hello world!I sing the aaaaasssaaasaaasaaasaa song.Aaaasssaaasaaasaaas


# Example of a context-free-grammar constraint

In [5]:
testCFG = ContextFreeGrammarConstraint()
testCFG.add_grammar_rule(testCFG.Rule("S", ["@i", "@f", "@ ", "E", "@ ", "@t", "@h", "@e", "@n", "@ ", "C"]))  # <S> -> if <E> then <C>
testCFG.add_grammar_rule(testCFG.Rule("E", ["@1"]))  # <E> -> 1|2|3|4|5|6|7|8|9|
testCFG.add_grammar_rule(testCFG.Rule("E", ["@2"]))
testCFG.add_grammar_rule(testCFG.Rule("E", ["@3"]))
testCFG.add_grammar_rule(testCFG.Rule("E", ["@4"]))
testCFG.add_grammar_rule(testCFG.Rule("E", ["@5"]))
testCFG.add_grammar_rule(testCFG.Rule("E", ["@6"]))
testCFG.add_grammar_rule(testCFG.Rule("E", ["@7"]))
testCFG.add_grammar_rule(testCFG.Rule("E", ["@8"]))
testCFG.add_grammar_rule(testCFG.Rule("E", ["@9"]))
testCFG.add_grammar_rule(testCFG.Rule("E", ["@(", "E", "@)"])) # <E> -> (<E>)
testCFG.add_grammar_rule(testCFG.Rule("E", ["E", "@+", "E"]))  # <E> -> <E> + <E>
testCFG.add_grammar_rule(testCFG.Rule("E", ["E", "@-", "E"]))  # <E> -> <E> - <E>
testCFG.add_grammar_rule(testCFG.Rule("E", ["E", "@*", "E"]))  # <E> -> <E> * <E>
testCFG.add_grammar_rule(testCFG.Rule("E", ["E", "@/", "E"]))  # <E> -> <E> / <E>
testCFG.add_grammar_rule(testCFG.Rule("C", ["@X", "@=", "E"])) # <C> -> X = <E>
testCFG.finalize_grammar()

In [7]:
llm = TransformersLLM("gpt2")

pn = PromptNode().set_model(llm)                  \
    .append("If 2+4 then return 8: ")             \
    .constrain(constraint=testCFG) \
    .gen(max_tokens=10)

text = pn.get_all_text()
print(text)

If 2+4 then return 8: if 2 then X=2
